In [1]:
# implement move generation functions
# currently in move_generation.py, needs to be updated if bitboard_implementation gets updated
# made using jupyter nbconvert --to script your_notebook.ipynb

import move_generation as mg
import random

In [2]:
# the starting game state, the convention in this engine is that the first 6 bitboards represent white, represented by 0
# white
white_pawn = int('0000000011111111000000000000000000000000000000000000000000000000', 2)
white_rook = int('1000000100000000000000000000000000000000000000000000000000000000', 2)
white_knight = int('0100001000000000000000000000000000000000000000000000000000000000', 2)
white_bishop = int('0010010000000000000000000000000000000000000000000000000000000000', 2)
white_queen = int('0000100000000000000000000000000000000000000000000000000000000000', 2)
white_king = int('0001000000000000000000000000000000000000000000000000000000000000', 2)

# black
black_pawn = int('0000000000000000000000000000000000000000000000001111111100000000', 2)
black_rook = int('0000000000000000000000000000000000000000000000000000000010000001', 2)
black_knight = int('0000000000000000000000000000000000000000000000000000000001000010', 2)
black_bishop = int('0000000000000000000000000000000000000000000000000000000000100100', 2)
black_queen = int('0000000000000000000000000000000000000000000000000000000000001000', 2)
black_king = int('0000000000000000000000000000000000000000000000000000000000010000', 2)

# en passant
white_en_passant = 0
black_en_passant = 0

# castling_rights
white_long_castle = True
white_short_castle = True
black_long_castle = True
black_short_castle = True

initial_game_state = [white_pawn, white_rook, white_knight, white_bishop, white_queen, white_king, black_pawn, black_rook, black_knight, black_bishop, black_queen, black_king, white_en_passant, black_en_passant, white_long_castle, white_short_castle, black_long_castle, black_short_castle]

In [3]:
color = 0
# generate pseudo-legal moves
plms = mg.move_generator(initial_game_state, color)

# turn pseudo-legal moves into legal moves
lms = mg.pseudo_to_legal(plms, initial_game_state, color)

# visualize
#for s in lms:
    #mg.print_board(s)

### Classical Monte Carlo Tree Search

https://www.youtube.com/watch?v=UXW2yZndl7U&t=771s

apply a value to each node using the upper confidence bound one applied to trees (UCT).

UCT for a state $S_i$: 
$UCT(S_i) = w_i/n_i + c*sqrt(ln(N_i)/n_i$

with $w_i$: the amount of wins
$n_i$: the amount of visits
$c$: exploration parameter
$N_i$: the amount of visists of the parent node

$w_i/n_i$ is the exploitation part

$c*sqrt(ln(N_i)/n_i$ is the exploration part

punish many iterations through the same nodes to encourage a more diverse exploration of the underlying nodes of a given state


1. calculate all possible moves from the current state
2. UCT will be infinite for all possible moves since none will have been visited yet
3. so we'll have to do a rollout for each leaf node (every possible move is a leaf node)
    - a rollout consists of doing random actions starting from the leaf node to a terminal state
    - a terminal state has a value (win or lose in chess)
3. add terminal value to the value of the initial leaf node (which had value 0 to start) and the initial state (also had value 0 to start)
4. recalculate UCT for all leaf nodes 

In [4]:
color = 0
# generate pseudo-legal moves
plms = mg.move_generator(initial_game_state, color)

# turn pseudo-legal moves into legal moves
lms = mg.pseudo_to_legal(plms, initial_game_state, color)


In [15]:
def evaluate_state(state, color):
    """
    state: gamestate to evaluate
    """
    plms = mg.move_generator(state, color)
    lms = mg.pseudo_to_legal(plms, state, color)
    
    if len(lms) == 0:
        return 1
    else:
        return 0
    
def change_player(x):
    return 1 if x == 0 else 0

In [16]:
class TreeNode:
    def __init__(self, state, q_value, visits, UCT, player_turn, parent = None):
        self.state = state
        self.q_value = q_value
        self.visits = visits
        self.UCT = UCT
        self.player_turn = player_turn
        self.children = []
        self.parent = parent
        
    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self
        
    def update_node(self, q_value):
        self.q_value += q_value
        self.visits += 1
        if self.parent != None:
            self.parent.update_node(q_value)
            self.UCT = calculate_UCT(self.q_value, self.visits, self.parent.visits)

In [17]:
def calculate_UCT(q_value, visits, parent_visits):
    return q_value/visits + 2*np.sqrt(np.log(visits)/parent_visits)

In [22]:
def player_MCTS(state):
    
    player_turn = 1
    
    init_value = 0
    init_visits = 0
    init_UCT = float('inf')

    root = TreeNode(state, init_value, init_visits, init_UCT, player_turn)

    for i in range(1000):
        #print(i)
        #always start at the root
        current_node = root

        #check if current node is leaf node, if not, go to the leaf node with the highest UCT
        while len(current_node.children) != 0:
            UCT = -float('inf')
            for child in current_node.children:
                if child.UCT > UCT:
                    current_node = child
                    UCT = child.UCT

        #check amount of visits
        #no visits, do a rollout
        if current_node.visits == 0:
            #print('rollout')
            rollout_player_turn = current_node.player_turn
            state = current_node.state
            q_value = 0
            rollout_length = 0
            while q_value == 0:
                plms = mg.move_generator(state, rollout_player_turn)
                moves = mg.pseudo_to_legal(plms, state, rollout_player_turn)
                random_move = random.randint(0, len(moves) - 1)
                state = moves[random_move]
                evaluation = evaluate_state(state, change_player(rollout_player_turn))
                if evaluation == 0:
                    q_value = 0
                elif player_turn == rollout_player_turn:
                    q_value = 1
                else:
                    q_value = -1
                
                rollout_player_turn = change_player(rollout_player_turn)
                rollout_length += 1
            #update leaf and all parent nodes
            #print('update')
            current_node.update_node(q_value)
        #if there are visits, expand
        else:
            #print('expand')
            plms = mg.move_generator(state, rollout_player_turn)
            moves = mg.pseudo_to_legal(plms, state, rollout_player_turn)
            for move in moves:
                #add all possible moves as leaves to the root
                child = TreeNode(move, 0, 0, float('inf'), change_player(rollout_player_turn))
                current_node.add_child(child)

    max_UCT = -float('inf')
    for child in root.children:
        if child.UCT > max_UCT:
            final_move = child.state
            max_UCT = child.UCT
            
    return final_move

    

In [26]:
def player_MCTS(state):
    
    player_turn = 0
    
    init_value = 0
    init_visits = 0
    init_UCT = float('inf')

    root = TreeNode(state, init_value, init_visits, init_UCT, player_turn)

    for i in range(100):
        #print(i)
        #always start at the root
        current_node = root

        #check if current node is leaf node, if not, go to the leaf node with the highest UCT
        while len(current_node.children) != 0:
            UCT = -float('inf')
            for child in current_node.children:
                if child.UCT > UCT:
                    current_node = child
                    UCT = child.UCT

        #check amount of visits
        #no visits, do a rollout
        if current_node.visits == 0:
            #print('rollout')
            rollout_player_turn = current_node.player_turn
            state = current_node.state
            q_value = 0
            rollout_length = 0
            while q_value == 0:
                plms = mg.move_generator(state, rollout_player_turn)
                moves = mg.pseudo_to_legal(plms, state, rollout_player_turn)
                random_move = random.randint(0, len(moves) - 1)
                state = moves[random_move]
                evaluation = evaluate_state(state, change_player(rollout_player_turn))
                if evaluation == 0:
                    q_value = 0
                elif player_turn == rollout_player_turn:
                    q_value = 1
                else:
                    q_value = -1
                
                rollout_player_turn = change_player(rollout_player_turn)
                rollout_length += 1
            #update leaf and all parent nodes
            #print('update')
            current_node.update_node(q_value)
        #if there are visits, expand
        else:
            #print('expand')
            plms = mg.move_generator(state, rollout_player_turn)
            moves = mg.pseudo_to_legal(plms, state, rollout_player_turn)
            for move in moves:
                #add all possible moves as leaves to the root
                child = TreeNode(move, 0, 0, float('inf'), change_player(rollout_player_turn))
                current_node.add_child(child)

    max_UCT = -float('inf')
    for child in root.children:
        if child.UCT > max_UCT:
            final_move = child.state
            max_UCT = child.UCT
            
    return final_move

    

In [27]:
def player_heuristic(state):
    
    """
    current heuristics:
    evaluate rollout by the player whose node is being rolled out, so the opponent is no longer assumed to take the worst option
    """
    
    player_turn = -1

    init_value = 0
    init_visits = 0
    init_UCT = float('inf')

    root = TreeNode(state, init_value, init_visits, init_UCT, player_turn)

    for i in range(100):
        #print(i)
        #always start at the root
        current_node = root

        #check if current node is leaf node
        while len(current_node.children) != 0:
            UCT = -float('inf')
            for child in current_node.children:
                if child.UCT > UCT:
                    current_node = child
                    UCT = child.UCT

        #check amount of visits
        #no visits, do a rollout
        if current_node.visits == 0:
            #print('rollout')
            rollout_player_turn = current_node.player_turn
            state = current_node.state
            q_value = 0
            rollout_length = 0
            while q_value == 0:
                plms = mg.move_generator(state, rollout_player_turn)
                moves = mg.pseudo_to_legal(plms, state, rollout_player_turn)
                random_move = random.randint(0, len(moves) - 1)
                state = moves[random_move]
                q_value = evaluate_state(state, current_node.player_turn)           #############################
                rollout_player_turn = -1*rollout_player_turn
                rollout_length += 1
            #update leaf and all parent nodes
            #print('update')
            current_node.update_node(q_value*current_node.player_turn*player_turn)                   ######################
        #if there are visits, expand
        else:
            #print('expand')
            for move in move_generator(current_node.state, current_node.player_turn):
                #add all possible moves as leaves to the root              
                #if a child leads to victory or loss, always choose that child
                init_value = 0
                child_moves = move_generator(move, -1*current_node.player_turn)
                for child_move in child_moves:
                    if evaluate_state(child_move, player_turn) == -1:
                        init_value += -1000
                init_value += evaluate_state(move, current_node.player_turn)
                child = TreeNode(move, init_value, 0, float('inf'), -1*current_node.player_turn)
                current_node.add_child(child)

    max_UCT = -float('inf')
    for child in root.children:
        if child.UCT > max_UCT:
            final_move = child.state
            max_UCT = child.UCT
            
    return final_move

In [ ]:
state = initial_game_state
outcomes = []
for i in range(10):
    go = 0
    state = initial_game_state
    while go == 0:
        state = player_MCTS(state)
        mg.print_board(state)
        go = 1*evaluate_state(state, 1)
        if evaluate_state(state, 1) == 1:
            print("1 wins")
            print(state)
        if go != 0:
            break
        state = player_MCTS2(state)
        mg.print_board(state)
        go = -1*evaluate_state(state, -1)
        if evaluate_state(state, -1) == 1:
            print("-1 wins")
            print(state)
        if go != 0:
            break
        print(state)
    outcomes.append(go)
    
    

Issues:
1. move generation function is slow, because it's written in python
2. move generation doesn't follow official notation
